# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#jupyter nbextension enable py gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

import os

# Import API key
from api_keys import g_key

In [2]:
import gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data = "../output_data/WeatherPy_Log.csv"
data_df= pd.read_csv(data)
data_df

,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
0,Lebu,CL,3883457,-37.62,-73.65,52.83,82,100,8.48
1,Rikitea,PF,4030556,-23.12,-134.97,75.20,84,100,30.02
2,Khatanga,RU,2022572,71.97,102.50,37.81,73,100,11.65
3,Hong Kong,HK,1819729,22.29,114.16,84.00,83,100,11.01
4,Skjervøy,NO,777682,70.03,20.97,54.19,71,0,14.67
...,...,...,...,...,...,...,...,...,...
575,Lagoa,PT,3372964,39.05,-27.98,63.23,73,18,6.62
576,Rafaela,AR,3839479,-31.25,-61.49,68.00,51,27,1.99
577,Atbasar,KZ,1526038,51.80,68.33,58.35,56,0,7.34
578,Portales,US,5484874,34.19,-103.33,91.99,29,1,18.34


In [4]:
data = pd.DataFrame.to_html(data_df)
data

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>City Name</th>\n      <th>Country</th>\n      <th>City ID</th>\n      <th>Latitude</th>\n      <th>Longitude</th>\n      <th>Max Temperature</th>\n      <th>Humidity</th>\n      <th>Cloudiness (%)</th>\n      <th>Wind Speed</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Lebu</td>\n      <td>CL</td>\n      <td>3883457</td>\n      <td>-37.62</td>\n      <td>-73.65</td>\n      <td>52.83</td>\n      <td>82</td>\n      <td>100</td>\n      <td>8.48</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Rikitea</td>\n      <td>PF</td>\n      <td>4030556</td>\n      <td>-23.12</td>\n      <td>-134.97</td>\n      <td>75.20</td>\n      <td>84</td>\n      <td>100</td>\n      <td>30.02</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Khatanga</td>\n      <td>RU</td>\n      <td>2022572</td>\n      <td>71.97</td>\n      <td>102.50</td>\n      <td>37.8

In [5]:
with open('Data.html', 'w') as fo:
    data_df.to_html(fo)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)
data_df = data_df.dropna()

locations = data_df[["Latitude", "Longitude"]].astype(float)
humidity = data_df["Humidity"].astype(float)
type(locations)

pandas.core.frame.DataFrame

In [13]:
! pip install -U gmaps
! jupyter nbextension enable --py --sys-prefix gmaps

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heat_layer)
fig


Requirement already up-to-date: gmaps in /Users/imacmattimacmatt/Applications/anaconda3/lib/python3.7/site-packages (0.9.0)


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
Hotel_df = data_df.loc[(data_df["Humidity"]<70)& (data_df["Max Temperature"]<55)& (data_df["Max Temperature"]
                                       >25) & (data_df["Wind Speed"]<5) & (data_df["Cloudiness (%)"]<10)]

Hotel_df

,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
9,Horasan,TR,745527,40.05,42.17,53.62,42,0,4.81
66,Provideniya,RU,4031574,64.38,-173.30,44.80,68,0,3.38
167,Elliot,ZA,1005781,-31.33,27.85,49.73,46,0,4.94
313,Altay,CN,1529651,47.87,88.12,40.95,66,0,3.33
477,Anadyr,RU,2127202,64.75,177.48,51.80,62,0,2.24
530,Evensk,RU,2125693,61.95,159.23,54.43,45,0,2.35


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# FIND THE HOTEL INFORMATION

#Create a Hotel Name List
Hotel_Name_List = []
#Find Hotels Using Google Maps nearby search
locations = Hotel_df[["Latitude", "Longitude"]].astype(float)
for x in range (len(locations)):
    lat=locations.iloc[x,0]
    long=locations.iloc[x,1]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
            "location": f"{lat},{long}",
            "radius": 5000,
            "keyword" : "hotel",
            "type": "hotel",
            "key": g_key,
    }

# Filter out Null Results
    response = requests.get(base_url, params=params).json()
    results=response["results"]
    try: 
        Hotel_Name = results[1]["name"]  
        Hotel_Name_List.append(Hotel_Name)
    except:
        print(f"{lat},{long}, No results for this location")
        
        No_Hotel = "No Hotel near this location"
        Hotel_Name_List.append(No_Hotel)

print(Hotel_Name_List)


64.38,-173.3, No results for this location
61.95,159.23, No results for this location
['Aydıner Resort Otel', 'No Hotel near this location', 'The Guardians Lodge', 'Altay Hengfeng Hotel', 'Motel Bar Hut ,, ,,', 'No Hotel near this location']


In [17]:
# Check the Hotel DataFrame
Hotel_df["Hotel Name"] = Hotel_Name_List
Hotel_df


/Users/imacmattimacmatt/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed,Hotel Name
9,Horasan,TR,745527,40.05,42.17,53.62,42,0,4.81,Aydıner Resort Otel
66,Provideniya,RU,4031574,64.38,-173.30,44.80,68,0,3.38,No Hotel near this location
167,Elliot,ZA,1005781,-31.33,27.85,49.73,46,0,4.94,The Guardians Lodge
313,Altay,CN,1529651,47.87,88.12,40.95,66,0,3.33,Altay Hengfeng Hotel
477,Anadyr,RU,2127202,64.75,177.48,51.80,62,0,2.24,"Motel Bar Hut ,, ,,"
530,Evensk,RU,2125693,61.95,159.23,54.43,45,0,2.35,No Hotel near this location


In [18]:
# Map the Top Six Hotels

Hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content= [f"Hotel Name: {name} " for name in Hotel_Name_List]
)


fig = gmaps.figure()
fig.add_layer(Hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add the Hotel marker layer on top of the heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(Hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

In [101]:
# THIS IS THE PART I TALKED TO MARK ABOUT AND THEN SKIPPED

# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City Name}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
# locations = Hotel_df[["Latitude", "Longitude"]]